<h1>Bengali.AI Handwritten Grapheme - Getting Started</h1>

# Introduction

Bengali is the 5th most spoken language in the world with hundreds of million of speakers. Optical character recognition is particularly challenging for Bengali. While Bengali has 49 letters (to be more specific 11 vowels and 38 consonants) in its alphabet, there are also 18 potential diacritics, or accents. This means that there are many more graphemes, or the smallest units in a written language. The added complexity results in ~13,000 different grapheme variations (compared to English’s 250 graphemic units).

Bangladesh-based non-profit Bengali.AI is focused on helping to solve this problem. They build and release crowdsourced, metadata-rich datasets and open source them through research competitions. Through this work, Bengali.AI hopes to democratize and accelerate research in Bengali language technologies and to promote machine learning education.

For this competition, we are given the image of a handwritten Bengali grapheme and are challenged to separately classify three constituent elements in the image: grapheme root, vowel diacritics, and consonant diacritics.

# Prepare for data analysis

## Load packages

In [ ]:
import os
import pandas as pd
import numpy as np
import PIL.Image
import time
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline 

## Check the data

We verify what data is available.

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

We have both csv files and parquet files.  
We will start by exploring csv files and will follow with parquet files.

# Data exploration

We start with the few csv files.

In [ ]:
DATA_FOLDER = '/kaggle/input/bengaliai-cv19/'
train_df = pd.read_csv(os.path.join(DATA_FOLDER, 'train.csv'))
train_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))
test_df.head()

In [ ]:
test_df.shape

In [ ]:
class_map_df = pd.read_csv(os.path.join(DATA_FOLDER, 'class_map.csv'))
class_map_df.head()

In [ ]:
class_map_df.shape

In [ ]:
sample_submission_df = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))
sample_submission_df.head()

In [ ]:
sample_submission_df.shape

We follow how with the parquet files. We will read only two of the parquet files for now, first train file.

In [ ]:
start_time = time.time()
train_0_df = pd.read_parquet(os.path.join(DATA_FOLDER,'train_image_data_0.parquet'))
print(f"`train_image_data_0` read in {round(time.time()-start_time,2)} sec.")                               

In [ ]:
train_0_df.shape

In [ ]:
train_0_df.head()

In [ ]:
start_time = time.time()
train_1_df = pd.read_parquet(os.path.join(DATA_FOLDER,'train_image_data_1.parquet'))
print(f"`train_image_data_1` read in {round(time.time()-start_time,2)} sec.")  

In [ ]:
train_1_df.shape

In [ ]:
train_1_df.head()

Each `train_image_data_x` (x = 0...3) contains **50210** rows and **32333** columns - size of each image being: **(137, 236)**. Totally there are **50210** x **4** = **200840** rows in the training set.  

We also read one of the test files.


In [ ]:
start_time = time.time()
test_0_df = pd.read_parquet(os.path.join(DATA_FOLDER,'test_image_data_0.parquet'))
print(f"`test_image_data_0` read in {round(time.time()-start_time,2)} sec.")  

In [ ]:
test_0_df.shape

In [ ]:
test_0_df.head()

## Unique values

We look here to the distribution of grapheme roots, vowel diacritics and consonant diacritics.

In [ ]:
print(f"Train: unique grapheme roots: {train_df.grapheme_root.nunique()}")
print(f"Train: unique vowel diacritics: {train_df.vowel_diacritic.nunique()}")
print(f"Train: unique consonant diacritics: {train_df.consonant_diacritic.nunique()}")
print(f"Train: total unique elements: {train_df.grapheme_root.nunique() + train_df.vowel_diacritic.nunique() + train_df.consonant_diacritic.nunique()}")
print(f"Class map: unique elements: \n{class_map_df.component_type.value_counts()}")
print(f"Total combinations: {pd.DataFrame(train_df.groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'])).shape[0]}")

## Data distribution

Let's start by viewing each grapheme.

Let's show the grapheme roots first.

In [ ]:
cm_gr = class_map_df.loc[(class_map_df.component_type=='grapheme_root'), 'component'].values
cm_vd = class_map_df.loc[(class_map_df.component_type=='vowel_diacritic'), 'component'].values  
cm_cd = class_map_df.loc[(class_map_df.component_type=='consonant_diacritic'), 'component'].values   

print(f"grapheme root:\n{15*'-'}\n{cm_gr}\n\n vowel discritic:\n{18*'-'}\n{cm_vd}\n\n consonant diacritic:\n{20*'-'}\n {cm_cd}")

Let's follow by investigating the most frequent values.

In [ ]:
def most_frequent_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    items = []
    vals = []
    for col in data.columns:
        itm = data[col].value_counts().index[0]
        val = data[col].value_counts().values[0]
        items.append(itm)
        vals.append(val)
    tt['Most frequent item'] = items
    tt['Frequence'] = vals
    tt['Percent from total'] = np.round(vals / total * 100, 3)
    return(np.transpose(tt))

Most frequent train values.

In [ ]:
most_frequent_values(train_df)

Most frequent test values.

In [ ]:
most_frequent_values(test_df)

Let's look now to the distribution of class values.

In [ ]:
def plot_count(feature, title, df, size=1):
    '''
    Plot count of classes of selected feature; feature is a categorical value
    param: feature - the feature for which we present the distribution of classes
    param: title - title to show in the plot
    param: df - dataframe 
    param: size - size (from 1 to n), multiplied with 4 - size of plot
    '''
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(df))
    g = sns.countplot(df[feature], order = df[feature].value_counts().index[:20], palette='Set3')
    g.set_title("Number and percentage of {}".format(title))
    if(size > 2):
        plt.xticks(rotation=90, size=8)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show() 

In [ ]:
plot_count('grapheme_root', 'grapheme_root (first most frequent 20 values - train)', train_df, size=4)

In [ ]:
plot_count('vowel_diacritic', 'vowel_diacritic (train)', train_df, size=3)

In [ ]:
plot_count('consonant_diacritic', 'consonant_diacritic (train)', train_df, size=3)

Let's show now distribution of combinations of features. We create a function to show a heatmap.

In [ ]:
def plot_count_heatmap(feature1, feature2, df, size=1):  
    '''
    Heatmap showing the distribution of couple of features
    param: feature1 - ex: vowel_diacritic
    param: feature2 - ex: consonant_diacritic
    '''
    tmp = train_df.groupby([feature1, feature2])['grapheme'].count()
    df = tmp.reset_index()
    df
    df_m = df.pivot(feature1, feature2, "grapheme")
    f, ax = plt.subplots(figsize=(9, size * 4))
    sns.heatmap(df_m, annot=True, fmt='3.0f', linewidths=.5, ax=ax)

Let's see first what consonant diacritics and vowel diacritics appears together.

In [ ]:
plot_count_heatmap('vowel_diacritic','consonant_diacritic', train_df)

We look now to the combinations of consonant diacritic and grapheme roots.

In [ ]:
plot_count_heatmap('grapheme_root','consonant_diacritic', train_df, size=8)

Here is the combinations of vowel diacritic and grapheme roots.

In [ ]:
plot_count_heatmap('grapheme_root','vowel_diacritic', train_df, size=8)

## Inspect grapheme images


We define a function to show a sample of size * size (ex: 5 x 5 = 25) handwritten graphemes.

In [ ]:
def display_image_from_data(data_df, size=5):
    '''
    Display grapheme images from sample data
    param: data_df - sample of data
    param: size - sqrt(sample size of data)
    '''
    plt.figure()
    fig, ax = plt.subplots(size,size,figsize=(12,12))
    # we show grapheme images for a selection of size x size samples
    for i, index in enumerate(data_df.index):
        image_id = data_df.iloc[i]['image_id']
        flattened_image = data_df.iloc[i].drop('image_id').values.astype(np.uint8)
        unpacked_image = PIL.Image.fromarray(flattened_image.reshape(137, 236))

        ax[i//size, i%size].imshow(unpacked_image)
        ax[i//size, i%size].set_title(image_id)
        ax[i//size, i%size].axis('on')

In [ ]:
display_image_from_data(train_0_df.sample(25))

We show also a sample from the second set of images and with fewer samples size (16).

In [ ]:
display_image_from_data(train_1_df.sample(16), size = 4)

Let's apply this function, this time to show not random graphemes, but the same grapheme, with different writing.   

For this we create a second function, to perform the sampling (based on variation of grapheme root, vowel diacritic and consonant diacritic, as parameters to the function).

In [ ]:
def display_writting_variety(data_df=train_0_df, grapheme_root=72, vowel_diacritic=0,\
                             consonant_diacritic=0, size=5):
    '''
    This function get a set of grapheme root, vowel diacritic and consonant diacritic
    and display a sample of 25 images for this grapheme
    param: data_df - the dataset used as source of data
    param: grapheme_root - the grapheme root label
    param: vowel_diacritic - the vowel diacritic label
    param: consonant_diacritic - the consonant diacritic label 
    param: size - sqrt(number of images to show)
    '''
    sample_train_df = train_df.loc[(train_df.grapheme_root == grapheme_root) & \
                                  (train_df.vowel_diacritic == vowel_diacritic) & \
                                  (train_df.consonant_diacritic == consonant_diacritic)]
    print(f"total: {sample_train_df.shape}")
    sample_df = data_df.merge(sample_train_df.image_id, how='inner')
    print(f"total: {sample_df.shape}")
    gr = sample_train_df.iloc[0]['grapheme']
    cm_gr = class_map_df.loc[(class_map_df.component_type=='grapheme_root')& \
                             (class_map_df.label==grapheme_root), 'component'].values[0]
    cm_vd = class_map_df.loc[(class_map_df.component_type=='vowel_diacritic')& \
                             (class_map_df.label==vowel_diacritic), 'component'].values[0]    
    cm_cd = class_map_df.loc[(class_map_df.component_type=='consonant_diacritic')& \
                             (class_map_df.label==consonant_diacritic), 'component'].values[0]    
    
    print(f"grapheme: {gr}, grapheme root: {cm_gr}, vowel discritic: {cm_vd}, consonant diacritic: {cm_cd}")
    sample_df = sample_df.sample(size * size)
    display_image_from_data(sample_df, size=size)

We apply the function for few combinations of grapheme root, vowel diacritic and consonant diacritic.

In [ ]:
display_writting_variety(train_0_df,72,1,1,4)

In [ ]:
display_writting_variety(train_0_df,64,1,2,4)

In [ ]:
display_writting_variety(train_1_df,13,0,0,4)

In [ ]:
display_writting_variety(train_1_df,23,3,2,4)

We can observe there is a large variety of writting for the selected graphemes.